In [29]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
arquivoRegiaoIBGE = '../Dengue/Regiao/regiao.xlsx'

In [34]:
dfRegiao = pd.read_excel(arquivoRegiaoIBGE, sheet_name='IBGE', usecols=[11,12,13], names=['geocode','nome_municipio','nome_regiao'])

In [35]:
dfRegiao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   geocode         92 non-null     int64 
 1   nome_municipio  92 non-null     object
 2   nome_regiao     92 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.3+ KB


In [37]:

ey_start = '2022'
ey_end = '2023'

for geocode,nome_municipio,nome_regiao in dfRegiao.itertuples(index=False):
    
    nome_municipio_t = nome_municipio.replace(' ','_')
    
    dfAPI = pd.read_csv(fr'https://info.dengue.mat.br/api/alertcity?geocode={geocode}&disease=dengue&format=csv&ew_start=1&ew_end=53&ey_start={ey_start}&ey_end={ey_end}')
   
    dfAPI['Nome_Município'] = nome_municipio
    dfAPI['codigo_municipio_completo'] = geocode
           
    arquivo = fr'../Dengue/Arquivos/arquivo_dengue_{geocode}_{nome_municipio_t}.csv'
    
    dfAPI.to_csv(arquivo, index=False, sep=';')

In [42]:
dfDengue = pd.DataFrame()

diretorio = '../Dengue/Arquivos/'

for diretorio, subpastas, arquivos in os.walk(diretorio):
    for arquivo in arquivos:
        path = os.path.join(os.path.realpath(diretorio), arquivo)
        
        dfParcial = pd.read_csv(path
                                , sep=';'
                                , usecols=[0,5,30,31]
                                ,names=['NumSemana','NumCasos','NomMunicipio','NumCodMunicipio']
                                ,header=0
                               )
        
        dfParcial['NumAnoMes'] = dfParcial['NumSemana'].str[:7]
        dfParcial['NumAno'] = dfParcial['NumSemana'].str[:4]
        dfParcial['NumMes'] = dfParcial['NumSemana'].str[5:7]
        
        dfDengue = dfDengue.append(dfParcial)

In [43]:
dfDengue

,NumSemana,NumCasos,NomMunicipio,NumCodMunicipio,NumAnoMes,NumAno,NumMes
0,2023-02-19,2,Angra dos Reis,3300100,2023-02,2023,02
1,2023-02-12,10,Angra dos Reis,3300100,2023-02,2023,02
2,2023-02-05,17,Angra dos Reis,3300100,2023-02,2023,02
3,2023-01-29,8,Angra dos Reis,3300100,2023-01,2023,01
4,2023-01-22,5,Angra dos Reis,3300100,2023-01,2023,01
...,...,...,...,...,...,...,...
55,2022-01-30,0,Volta Redonda,3306305,2022-01,2022,01
56,2022-01-23,0,Volta Redonda,3306305,2022-01,2022,01
57,2022-01-16,1,Volta Redonda,3306305,2022-01,2022,01
58,2022-01-09,0,Volta Redonda,3306305,2022-01,2022,01


In [44]:
dfDengue = pd.merge(left=dfDengue
                              , right=dfRegiao
                              , left_on='NumCodMunicipio'
                              , right_on='geocode'                                                                                              
                                    )[['NumAnoMes','NumAno','NumMes','NumSemana','NomMunicipio','nome_regiao','NumCasos']]

In [52]:
dfDengue[dfDengue['NumAno']=='2023'].groupby(['NumSemana']).agg(QtdCasos=('NumCasos','sum'))

,QtdCasos
NumSemana,
2023-01-01,577
2023-01-08,628
2023-01-15,678
2023-01-22,804
2023-01-29,977
2023-02-05,1106
2023-02-12,516
2023-02-19,111


In [18]:
dfDengue.info()

0,5,30,31

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5520 entries, 0 to 59
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   data_iniSE                 5520 non-null   object 
 1   SE                         5520 non-null   int64  
 2   casos_est                  5520 non-null   float64
 3   casos_est_min              5520 non-null   int64  
 4   casos_est_max              5509 non-null   float64
 5   casos                      5520 non-null   int64  
 6   p_rt1                      5520 non-null   float64
 7   p_inc100k                  5520 non-null   float64
 8   Localidade_id              5520 non-null   int64  
 9   nivel                      5520 non-null   int64  
 10  id                         5520 non-null   int64  
 11  versao_modelo              5520 non-null   object 
 12  tweet                      5520 non-null   float64
 13  Rt                         5520 non-null   float64